In [ ]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
#import warnings
#warnings.filterwarnings("ignore")

from nupack import *
from Bio import pairwise2
from Bio.Seq import Seq
from Bio.SeqUtils import GC, molecular_weight

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, \
                             AdaBoostRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression, ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import sklearn.gaussian_process as gp
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
CONFIG = {
    'path': 'datasets/rna-logic-gates',
    'remove_control': True,
    'remove_shared_sequence': True,
    'logscale': True,
    'use_median': True, #otherwise use mean
    'model_nostacking': True
}

In [ ]:
def get_kmers(sequence, size=3):
    return [sequence[x:x+size].upper() for x in range(len(sequence)-size+1)]

def generate_seq_features(data, kmers):
    
    feats = pd.DataFrame()
    labels = []
    for k in kmers:
        words5 = data['rna5'].apply(lambda x: get_kmers(str(x), k))
        words3 = data['rna3'].apply(lambda x: get_kmers(str(x), k))
        texts = list(words5 + words3)
        for i in range(len(texts)):
            texts[i] = ' '.join(texts[i])

        cv = CountVectorizer()
        cv.fit(texts)
        seq_feats = pd.DataFrame(cv.transform(texts).toarray())
        feats = pd.concat([feats, seq_feats], axis=1)
        feats.columns = np.arange(feats.shape[1])
        labels.extend(cv.get_feature_names_out())
    
    return feats, labels

In [ ]:
def preprocess_data(raw_data, config=CONFIG):
    
    data = raw_data.copy()
    if config['remove_shared_sequence']:
        data['rna5'] = data['rna5'].apply(lambda x: x.replace('T', 'U')[577:-49]) #hardcoded position for the non-shared region of 5-egs
        data['rna3'] = data['rna3'].apply(lambda x: x.replace('T', 'U')[68:-732]) #hardcoded position for the non-shared region of 3-egs
    data['rna5_rna3'] = data['rna5'] + data['rna3']
    if config['use_median']:
        data['fluo'] = data[['fluo1', 'fluo2', 'fluo3']].median(axis=1)
    else:
        data['fluo'] = data[['fluo1', 'fluo2', 'fluo3']].mean(axis=1)
    if config['logscale']:
        data['fluo'] = np.log10(data['fluo'])
    data['len5'] = data['rna5'].apply(lambda x: len(x))
    data['len3'] = data['rna3'].apply(lambda x: len(x))
    if config['remove_control']:
        data = data[data['len5']>0].reset_index(drop=True) #remove control because controls do not have non-shared 5-egs
    
    return data.drop(['fluo1', 'fluo2', 'fluo3'], axis=1)

In [ ]:
raw_data = pd.read_csv('{}/egs.csv'.format(CONFIG['path']))
data = preprocess_data(raw_data)
data.shape

In [ ]:
def generate_thermo_features(data, config=CONFIG, encode_non_numeric=True):
    
    models = [Model(material='RNA', ensemble='stacking')]
    if config['model_nostacking']:
        models.append(Model(material='RNA', ensemble='nostacking'))
        
    for i, model in tqdm(enumerate(models)):
        arr = [pfunc(strands=[str1, str2], model=model) \
               for str1, str2 in zip(data['rna5'].tolist(), data['rna3'].tolist())]
        data['pfunc-0_{}'.format(i)] = [x[0] for x in arr]
        data['pfunc-1_{}'.format(i)] = [x[1] for x in arr]
        
        data['pfunc-0_{}'.format(i)] = data['pfunc-0_{}'.format(i)].astype(float)
        
    for i, model in tqdm(enumerate(models)):
        arr = [pairs(strands=[str1, str2], model=model) \
               for str1, str2 in zip(data['rna5'].tolist(), data['rna3'].tolist())]
        data['prob_matrix_{}'.format(i)] = [x.to_array().ravel() for x in arr]
 
    for i, model in tqdm(enumerate(models)):
        arr = [mfe(strands=[str1, str2], model=model) \
               for str1, str2 in zip(data['rna5'].tolist(), data['rna3'].tolist())]
        data['mfe_energy_{}'.format(i)] = [x[0].energy for x in arr]
        data['mfe_stack_energy_{}'.format(i)] = [x[0].stack_energy for x in arr]
        data['mfe_structure_{}'.format(i)] = [str(x[0].structure) for x in arr]
        data['mfe_matrix_{}'.format(i)] = [x[0].structure.matrix().ravel() for x in arr]
        data['mfe_pairlist_{}'.format(i)] = [x[0].structure.pairlist() for x in arr]
        data['mfe_nicks_{}'.format(i)] = [x[0].structure.nicks() for x in arr]
        
    for i, model in tqdm(enumerate(models)):
        arr = [structure_energy(strands=[str1, str2], structure=structure, model=model) \
               for str1, str2, structure in zip(data['rna5'].tolist(), data['rna3'].tolist(), \
               data['mfe_structure_{}'.format(i)])]
        data['struct_dg_{}'.format(i)] = arr
    
    for i, model in tqdm(enumerate(models)):
        arr = [structure_probability(strands=[str1, str2], structure=structure, model=model) \
               for str1, str2, structure in zip(data['rna5'].tolist(), data['rna3'].tolist(), \
               data['mfe_structure_{}'.format(i)])]
        data['struct_proba_{}'.format(i)] = arr

    for i, model in tqdm(enumerate(models)):
        arr = [subopt(strands=[str1, str2], energy_gap=1.5, model=model) \
               for str1, str2 in zip(data['rna5'].tolist(), data['rna3'].tolist())]
        data['subopt_energy_{}'.format(i)] = [x[0].energy for x in arr]
        data['subopt_stack_energy_{}'.format(i)] = [x[0].stack_energy for x in arr]
        
    for i, model in tqdm(enumerate(models)):
        arr = [ensemble_size(strands=[str1, str2], model=model) \
               for str1, str2 in zip(data['rna5'].tolist(), data['rna3'].tolist())]
        data['ensemble_{}'.format(i)] = arr
    
    try:
        for i, model in tqdm(enumerate(models)):
            arr = [defect(strands=[str1, str2], structure=structure, model=model) \
                   for str1, str2, structure in zip(data['rna5'].tolist(), data['rna3'].tolist(), \
                   data['mfe_structure_{}'.format(i)])]
            data['defect_{}'.format(i)] = arr
    except:
        print('***DEFECT does not work***')

    data['distance'] = [seq_distance(str1, str2)
                        for str1, str2 in zip(data['rna5'].tolist(), data['rna3'].tolist())]
    data['global_alg'] = [pairwise2.align.globalxx(str1, str2, score_only=True)
                          for str1, str2 in zip(data['rna5'].tolist(), data['rna3'].tolist())]
    data['local_alg'] = [pairwise2.align.localxx(str1, str2, score_only=True)
                         for str1, str2 in zip(data['rna5'].tolist(), data['rna3'].tolist())]

    data['gc5'] = [GC(x) for x in data['rna5']]
    data['gc3'] = [GC(x) for x in data['rna3']]
    data['weight5'] = [molecular_weight(x, 'RNA') for x in data['rna5']]
    data['weight3'] = [molecular_weight(x, 'RNA') for x in data['rna3']]
    data['C5'] = [Seq(x).count('C') for x in data['rna5']]
    data['C3'] = [Seq(x).count('C') for x in data['rna3']]
    data['G5'] = [Seq(x).count('G') for x in data['rna5']]
    data['G3'] = [Seq(x).count('G') for x in data['rna3']]
    data['U5'] = [Seq(x).count('U') for x in data['rna5']]
    data['U3'] = [Seq(x).count('U') for x in data['rna3']]
    data['A5'] = [Seq(x).count('A') for x in data['rna5']]
    data['A3'] = [Seq(x).count('A') for x in data['rna3']]
    
    if encode_non_numeric:
        #dealing with non numeric features
        for i, model in tqdm(enumerate(models)):
            temp = pd.DataFrame(data['prob_matrix_{}'.format(i)].tolist())
            temp.columns = ['pme_{}_{}'.format(i, j) for j in np.arange(temp.shape[1])]
            temp.drop([col for col, val in temp.sum().iteritems() if val==0], axis=1, inplace=True)
            data.drop('prob_matrix_{}'.format(i), axis=1, inplace=True)
            data = pd.concat([data, temp], axis=1)

        for i, model in tqdm(enumerate(models)):
            temp = pd.DataFrame(data['mfe_matrix_{}'.format(i)].tolist())
            temp.columns = ['mme_{}_{}'.format(i, j) for j in np.arange(temp.shape[1])]
            temp.drop([col for col, val in temp.sum().iteritems() if val==0 or val==1296], axis=1, inplace=True)
            data.drop('mfe_matrix_{}'.format(i), axis=1, inplace=True)
            data = pd.concat([data, temp], axis=1)

        for i, model in tqdm(enumerate(models)):
            temp = pd.DataFrame(data['mfe_pairlist_{}'.format(i)].tolist())
            temp.columns = ['mpe_{}_{}'.format(i, j) for j in np.arange(temp.shape[1])]
            data.drop('mfe_pairlist_{}'.format(i), axis=1, inplace=True)
            data = pd.concat([data, temp], axis=1)

        for i, model in tqdm(enumerate(models)):
            temp = pd.DataFrame(data['mfe_nicks_{}'.format(i)].tolist())
            temp.columns = ['mne_{}_{}'.format(i, j) for j in np.arange(temp.shape[1])]
            data.drop('mfe_nicks_{}'.format(i), axis=1, inplace=True)
            data = pd.concat([data, temp], axis=1)

        for i, model in tqdm(enumerate(models)):
            data['mfe_structure_{}_dot'.format(i)] = data['mfe_structure_{}'.format(i)].apply(lambda x: x.count('.'))
            data['mfe_structure_{}_left_bracket'.format(i)] = data['mfe_structure_{}'.format(i)].apply(lambda x: x.count('('))
            data['mfe_structure_{}_plus'.format(i)] = data['mfe_structure_{}'.format(i)].apply(lambda x: x.count('+'))
            data['mfe_structure_{}_right_bracket'.format(i)] = data['mfe_structure_{}'.format(i)].apply(lambda x: x.count(')'))
            data.drop('mfe_structure_{}'.format(i), axis=1, inplace=True)
    else:
        for i, model in tqdm(enumerate(models)):
            data.drop(['prob_matrix_{}'.format(i), 'mfe_matrix_{}'.format(i), 'mfe_pairlist_{}'.format(i),
                       'mfe_nicks_{}'.format(i), 'mfe_structure_{}'.format(i)], axis=1, inplace=True)
      
    return data.drop(['rna5', 'rna3', 'rna5_rna3'], axis=1)

Feature engineering using thermodynamics features.

In [ ]:
thermo_feats = generate_thermo_features(data, encode_non_numeric=True)
thermo_feats.shape

Evaluating multiple regressors.

In [ ]:
#all the models
regressors = [
    RandomForestRegressor(),
    GradientBoostingRegressor(),
    AdaBoostRegressor(),
    HistGradientBoostingRegressor(),
    XGBRegressor(),
    CatBoostRegressor(),
    LGBMRegressor(),
    MLPRegressor(),
    SGDRegressor(),
    SVR(),
    LinearRegression(),
    ElasticNet(),
    KernelRidge(),
    BayesianRidge()
]

5-fold cross-validation.

In [ ]:
X = thermo_feats.drop('fluo', axis=1)
y = thermo_feats['fluo']
kf = KFold(n_splits=5, shuffle=True)
est_performances_thermo = []
 
for estimator in tqdm(regressors):
    
    r2_scores = []
    y_tests = []
    y_preds = []

    for i, (train_index, test_index) in tqdm(enumerate(kf.split(X))):

        X_train = thermo_feats.loc[train_index].drop('fluo', axis=1).values
        X_test = thermo_feats.loc[test_index].drop('fluo', axis=1).values
        y_train = thermo_feats.loc[train_index]['fluo'].values
        y_test = thermo_feats.loc[test_index]['fluo'].values
        
        estimator.fit(X_train, y_train)
        y_pred = estimator.predict(X_test)
        r2_scores.append(r2_score(y_test, y_pred))
        y_tests.append(y_test.tolist())
        y_preds.append(y_pred.tolist())
        
    est_performances_thermo.append((r2_scores, y_tests, y_preds))

In [ ]:
for e in est_performances_thermo:
    print(np.mean(e[0]), np.std(e[0]))

Feature engineering using sequence features.

In [ ]:
seq_feats, seq_feats_label = generate_seq_features(data, kmers=[3, 4, 5, 6])
seq_feats.shape

In [ ]:
X = seq_feats.copy()
y = thermo_feats['fluo']
kf = KFold(n_splits=5, shuffle=True)
est_performances_seq = []
 
for estimator in tqdm(regressors):
    
    r2_scores = []
    y_tests = []
    y_preds = []

    for i, (train_index, test_index) in tqdm(enumerate(kf.split(X))):

        X_train = seq_feats.loc[train_index].values
        X_test = seq_feats.loc[test_index].values
        y_train = thermo_feats.loc[train_index]['fluo'].values
        y_test = thermo_feats.loc[test_index]['fluo'].values
        
        estimator.fit(X_train, y_train)
        y_pred = estimator.predict(X_test)
        r2_scores.append(r2_score(y_test, y_pred))
        y_tests.append(y_test.tolist())
        y_preds.append(y_pred.tolist())
        
    est_performances_seq.append((r2_scores, y_tests, y_preds))

In [ ]:
for e in est_performances_seq:
    print(np.mean(e[0]), np.std(e[0]))

Feature engineering using combined features.

In [ ]:
X = pd.concat([thermo_feats, seq_feats], axis=1)

kf = KFold(n_splits=5, shuffle=True)
est_performances_combo = []
 
for estimator in tqdm(regressors):
    
    r2_scores = []
    y_tests = []
    y_preds = []

    for i, (train_index, test_index) in tqdm(enumerate(kf.split(X))):

        X_train = X.loc[train_index].drop('fluo', axis=1).values
        X_test = X.loc[test_index].drop('fluo', axis=1).values
        y_train = X.loc[train_index]['fluo'].values
        y_test = X.loc[test_index]['fluo'].values
        
        estimator.fit(X_train, y_train)
        y_pred = estimator.predict(X_test)
        r2_scores.append(r2_score(y_test, y_pred))
        y_tests.append(y_test.tolist())
        y_preds.append(y_pred.tolist())
        
    est_performances_combo.append((r2_scores, y_tests, y_preds))

In [ ]:
for e in est_performances_combo:
    print(np.mean(e[0]), np.std(e[0]))

Performance reports.

In [ ]:
regressor_name = [
    "Random Forest",
    "Gradient Boosting",
    "Adaptive Boosting",
    "Histogram-based Gradient Boosting",
    "Extreme Gradient Boosting",
    "Category Boosting",
    "Light Gradient Boosting",
    "Neural Network (Multi Layer Perceptron)",
    "Stochastic Gradient Descent",
    "Support Vector Regression",
    "Linear Regression",
    "Elastic Net Regression",
    "Kernel Ridge Regression",
    "Bayesian Ridge Regression"
]
df = pd.DataFrame()
temp = pd.DataFrame([(np.mean(e[0]), np.std(e[0])) for e in est_performances_combo], columns=['avg_r2', 'std_r2'])
temp['model'] = regressor_name
temp = temp[['model', 'avg_r2', 'std_r2']]
df = pd.concat([df, temp], axis=0)
df.to_csv('{}/model/performances_combined.csv'.format(CONFIG['path']), index=False)

In [ ]:
for x in tqdm(range(len(regressor_name))):
    df = pd.DataFrame()
    for y in range(5):
        temp = pd.DataFrame([(regressor_name[x], a, b, y+1) for a, b in zip(est_performances_seq[x][1][y], est_performances_seq[x][2][y])],
                           columns=['model', 'experiment', 'prediction', 'split #'])
        df = pd.concat([df, temp], axis=0)
    df.to_csv('{}/model/sequence-motifs/{}.csv'.format(CONFIG['path'], regressor_name[x]), index=False)